In [11]:
import random

random.seed(42)

processing_times = [4, 2, 8, 3, 6, 7, 5, 10, 12, 1, 9]  # 11 jobs
num_jobs = len(processing_times)
num_machines = 4  # more machines

num_ants = 20
num_iterations = 10
alpha = 1.0
beta = 2.0
rho = 0.1
Q = 100

pheromone = [[0.1 for _ in range(num_machines)] for _ in range(num_jobs)]

def calculate_makespan(assignment, processing_times, num_machines):
    machine_loads = [0] * num_machines
    for job, machine in enumerate(assignment):
        machine_loads[machine] += processing_times[job]
    return max(machine_loads), machine_loads

def select_machine_for_job(job, pheromone, processing_times):
    pheromone_power = [pheromone[job][m] ** alpha for m in range(num_machines)]
    heuristic_power = [(1.0 / processing_times[job]) ** beta for _ in range(num_machines)]

    total = sum(p * h for p, h in zip(pheromone_power, heuristic_power))
    probabilities = [(p * h) / total for p, h in zip(pheromone_power, heuristic_power)]

    r = random.random()
    cumulative = 0
    for machine, prob in enumerate(probabilities):
        cumulative += prob
        if r <= cumulative:
            return machine
    return num_machines - 1

best_global_assignment = None
best_global_makespan = float('inf')

for iteration in range(1, num_iterations + 1):
    best_iteration_assignment = None
    best_iteration_makespan = float('inf')
    
    for ant in range(num_ants):
        assignment = [-1] * num_jobs
        for job in range(num_jobs):
            machine = select_machine_for_job(job, pheromone, processing_times)
            assignment[job] = machine
        makespan, _ = calculate_makespan(assignment, processing_times, num_machines)

        if makespan < best_iteration_makespan:
            best_iteration_makespan = makespan
            best_iteration_assignment = assignment
    
    if best_iteration_makespan < best_global_makespan:
        best_global_makespan = best_iteration_makespan
        best_global_assignment = best_iteration_assignment

    # Evaporation
    for job in range(num_jobs):
        for m in range(num_machines):
            pheromone[job][m] *= (1 - rho)

    # Deposit pheromone on best local assignment
    deposit = Q / best_iteration_makespan
    for job, machine in enumerate(best_iteration_assignment):
        pheromone[job][machine] += deposit

    # Show scheduling details for this iteration
    print(f"Iteration {iteration}: Best makespan this iteration = {best_iteration_makespan}, Global best makespan = {best_global_makespan}")
    print("Job assignments (job: machine):")
    for job, machine in enumerate(best_iteration_assignment):
        print(f"  Job {job} (time {processing_times[job]}): Machine {machine}")
    _, loads = calculate_makespan(best_iteration_assignment, processing_times, num_machines)
    print(f"Machine loads: {loads}")
    print("-" * 40)

print("\nFinal best assignment:")
print("Job assignments (job: machine):")
for job, machine in enumerate(best_global_assignment):
    print(f"Job {job} (time {processing_times[job]}): Machine {machine}")
_, loads = calculate_makespan(best_global_assignment, processing_times, num_machines)
print(f"Machine loads: {loads}")
print(f"Final makespan: {best_global_makespan}")


Iteration 1: Best makespan this iteration = 22, Global best makespan = 22
Job assignments (job: machine):
  Job 0 (time 4): Machine 0
  Job 1 (time 2): Machine 0
  Job 2 (time 8): Machine 1
  Job 3 (time 3): Machine 1
  Job 4 (time 6): Machine 0
  Job 5 (time 7): Machine 3
  Job 6 (time 5): Machine 3
  Job 7 (time 10): Machine 1
  Job 8 (time 12): Machine 2
  Job 9 (time 1): Machine 1
  Job 10 (time 9): Machine 3
Machine loads: [12, 22, 12, 21]
----------------------------------------
Iteration 2: Best makespan this iteration = 19, Global best makespan = 19
Job assignments (job: machine):
  Job 0 (time 4): Machine 0
  Job 1 (time 2): Machine 0
  Job 2 (time 8): Machine 1
  Job 3 (time 3): Machine 3
  Job 4 (time 6): Machine 0
  Job 5 (time 7): Machine 0
  Job 6 (time 5): Machine 3
  Job 7 (time 10): Machine 1
  Job 8 (time 12): Machine 2
  Job 9 (time 1): Machine 1
  Job 10 (time 9): Machine 3
Machine loads: [19, 19, 12, 17]
----------------------------------------
Iteration 3: Best ma